In [18]:
import pandas as pd, numpy as np
import nltk

from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

In [19]:
!pip3 install pandas scikit-learn numpy nltk hyperopt
import pandas as pd
import numpy as np
import nltk

You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
train = pd.read_csv('datasets/Train.csv')

In [21]:
test = pd.read_csv('datasets/Test.csv')

In [22]:
train.head()

,reviewerID,amazon-id,helpful,unixReviewTime,reviewText,overall,reviewTime,summary,price,categories,root-genre,title,artist,label,first-release-year,songs,salesRank,related
0,-4984057859803657856,1877521326299865484,"[2, 2]",1302739200,Very nice music for practicing my Tai Chi. I d...,4,"04 14, 2011",Beautiful,16.47,"['CDs & Vinyl', 'New Age']",New Age,-3267874170410107454,-7180760356347753735,Cdbaby/Cdbaby,NaN,"[7058439142327364074, 6037075874942075284, 852...",27222,"{'also_bought': [-404470919165672227, 11968160..."
1,9136764282801708742,1877521326299865484,"[11, 11]",1180396800,I recently starting doing Tai Chi which I love...,5,"05 29, 2007",Tranquillity In Motion !!!,16.47,"['CDs & Vinyl', 'New Age']",New Age,-3267874170410107454,-7180760356347753735,Cdbaby/Cdbaby,NaN,"[7058439142327364074, 6037075874942075284, 852...",27222,"{'also_bought': [-404470919165672227, 11968160..."
2,2164551966908582519,1877521326299865484,"[0, 0]",1361404800,My wife uses it for her class room the kids lo...,5,"02 21, 2013",Great Stuff,16.47,"['CDs & Vinyl', 'New Age']",New Age,-3267874170410107454,-7180760356347753735,Cdbaby/Cdbaby,NaN,"[7058439142327364074, 6037075874942075284, 852...",27222,"{'also_bought': [-404470919165672227, 11968160..."
3,-7309200698931694843,1877521326299865484,"[4, 4]",1338163200,We bought this music to go Dr Lam DVD. The mus...,5,"05 28, 2012",Beautiful,16.47,"['CDs & Vinyl', 'New Age']",New Age,-3267874170410107454,-7180760356347753735,Cdbaby/Cdbaby,NaN,"[7058439142327364074, 6037075874942075284, 852...",27222,"{'also_bought': [-404470919165672227, 11968160..."
4,-4461682407031037732,1877521326299865484,"[0, 0]",1396310400,It helps me do my exercise because it sets the...,5,"04 1, 2014",tai chi music,16.47,"['CDs & Vinyl', 'New Age']",New Age,-3267874170410107454,-7180760356347753735,Cdbaby/Cdbaby,NaN,"[7058439142327364074, 6037075874942075284, 852...",27222,"{'also_bought': [-404470919165672227, 11968160..."


In [43]:
"""
handle_text: takes out stop words and punctuation, lemmatizes, and converts to lowercase
* input: string
* output: hanlded string
"""
def handle_text(text):
    # Check that argument provided is a string
    if not isinstance(text,str):
        return ""
    output = []
    stop_words = set(stopwords.words('english'))
    # use stemmer instead of lemmatizer here
    # wordnet_lemmatizer = WordNetLemmatizer()
    sno = nltk.stem.SnowballStemmer('english')
    for word in nltk.word_tokenize(text): 
        # Disclude stop words and words with punctuation
        word = word.lower()
        if word not in stop_words and word.isalpha():
            # Add lemmatized words
            #output.append(wordnet_lemmatizer.lemmatize(word))
            output.append(sno.stem(word))
    return " ".join(output) # return the list of words as one single string

In [44]:
"""
convert_text: applies handle_text to all summaries and reviews in dataframe
* input: dataframe
* output: new dataframe with converted summaries/reviews
"""
def convert_text(df):
    df = df.assign(summary=df['summary'].apply(handle_text))
    df = df.assign(reviewText=df['reviewText'].apply(handle_text))
    return df

In [45]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

In [46]:
"""
prep_df: applies convert text to both summary and review features
* input: dataframe
* output: new dataframe with converted summaries/reviews
"""

def prep_df(df):
    # Convert to lowercase, remove punctuation and stop words, lemmatize words
    # df = convert_text(df)
    # uncommon_words = get_uncommon_words(df['reviewText'], 2)
    # Remove uncommon words
    #return remove_words_reviews(df, uncommon_words)
    return convert_text(df)

In [47]:
"""
awesome_products: extracts the training label based on overall ratings of the products in the dataset
* input: dataframe
* output: the training label - y_train
"""

def awesome_products(df):
    df = df.groupby('amazon-id').agg({'overall': lambda x: 1 if np.mean(x) > 4.5 else 0})
    return df['overall']

In [48]:
"""
make_prediction: uses LinearSVC as the base model to predictions based on training data
* input: x_train, y_train, x_test
* output: the model  used and the predictions from the model
"""

def make_prediction(x_train, y_train, x_test):
    print("Predicting")
    model = LinearSVC()
    model.fit(x_train, y_train)
    print("Done")
    return model, model.predict(x_test)

In [49]:
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, f1_score, classification_report

def test_model():
    
    # gets the training vector and the training label based on the training data
    train_vec, awe_vec, ct = transform_train(train)
    
    # splits the data into the training set and the validation set
    x_train, x_test, y_train, y_test = train_test_split(train_vec, awe_vec)
    
    #x_train_new , y_train_new = correct_oversampling(x_train, y_train)
    
    # gets predictions
    
    model, y_pred = make_prediction(x_train, y_train, x_test)
    # print("f1_score=", f1_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))  

In [50]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

"""
transform_train: Allows for feature extraction from multiple columns in the training data using a ColumnTransformer 
object. The transformer object needs to be told how to transform the columns we're interested in. ct_opt fulfills this
purpose. It returns the transformed training data as a vector along with the training label, using the awesome_products 
method and the column transformer object. This makes sure that we use the same vectorizers to transform summary and review text in the
testing data.
* input: data
* output: the training vector, y label and the column transformer object
"""

def transform_train(data): 
    print("Prepping train df")
    data_new = prep_df(data)
    print("Done")
    combined_df = data_new.groupby('amazon-id').agg({'reviewText': ' '.join, 'summary': ' '.join, 'salesRank': lambda x: np.mean(x), 'price': lambda x: np.mean(x)})
    print("Feature extraction")
    # change vectorizer!
    # improving max features
    # using stem instead of lemmatizer 
    
    # Column Transformer options - this list specifies how the columns we're interested in are transformed
    # reviewText gets a tfidf vectorizer
    # summary gets a tfidf vectorizer too (so that we don't use the same vectorizer for both features)
    # salesRank and price are both normalized
    
    ct_opts = [('reviewText_bow', TfidfVectorizer(max_features = 20000, min_df=2), 'reviewText'),
          ('summary_bow', TfidfVectorizer(max_features = 20000, min_df=2), 'summary'),
          ('salesRank_norm', MinMaxScaler(), ['salesRank']),
          ('price_norm', MinMaxScaler(), ['price'])]
    
    # The ColumnTransformer object changes the training data in accordance with the options specified above. 
    # the other features in the dataset are disregarded
    ct = ColumnTransformer(ct_opts, remainder = 'drop')
    
    # fitting the training data based on the column transformer object
    res_vec = np.array(ct.fit_transform(combined_df).toarray())
    print("Done")
    print("Prepping y labels")
    awe_vec = awesome_products(data_new)
    print("Done")
    
    # Passing ct here so that we can use the same vectorizers on the testing data
    return res_vec, awe_vec, ct 

In [51]:
"""
transform_test: Allows for feature extraction from multiple columns in the testing data using a ColumnTransformer 
object. We use the same transformer object we got from transform train here using the input parameter ct.
* input: testing data, the previously used ColumnTransformer 
* output: the testing data transformed into a vector
"""

def transform_test(data, ct):
    print("Prepping test data")
    data_new = prep_df(data)
    print("Done")
    print("Feature extraction")
    combined_df = data_new.groupby('amazon-id').agg({'reviewText': ' '.join, 'summary': ' '.join, 'salesRank': lambda x: np.mean(x), 'price': lambda x: np.mean(x)})
    #ct_opts = [('reviewText_bow', TfidfVectorizer(max_features = 8000, min_df=2), 'reviewText'),
    #      ('summary_bow', TfidfVectorizer(max_features = 8000, min_df=2), 'summary'),
    #     ('salesRank_norm', MinMaxScaler(), ['salesRank']),
    #      ('price_norm', MinMaxScaler(), ['price'])]
    # ct = ColumnTransformer(ct_opts, remainder = 'drop')
    print("Done")
    res_vec = np.array(ct.transform(combined_df).toarray())
    return res_vec

In [52]:
def run_model():
    
    # gets the training vector and the training label based on the training data
    df_train = pd.read_csv('datasets/Train.csv')
    x_train, y_train, ct = transform_train(df_train)

    # gets the test vector based on the testing data
    df_test = pd.read_csv('datasets/Test.csv')
    x_test = transform_test(df_test, ct)

    
    # makes predictions
    model, pred_y = make_prediction(train_vec, train_y, test_vec)
    
    #saves the predictions to a .csv file - check output before submitting code
    save_results(df_test, pred_y)
    
    

In [53]:
# gets the training vector and the training label based on the training data

def save_results(df_test, pred_y):
    print("Writing output")
    output = pd.DataFrame({'amazon-id': df_test["amazon-id"].drop_duplicates().reset_index(drop=True), 'Awesome': pred_y})
    output.to_csv('predictions_test.csv', index= False)
    output.head(10)
    print("Done")
    
    

In [ ]:
test_model()

Prepping train df


In [60]:
run_model()
# The code after this point is stuff I've been playing around with
# a: sentiment analysis
# b: oversampling class 0 using SMOTE


Prepping train df
Done
Feature extraction
Done
Prepping y labels
Done
Prepping test data
Done
Feature extraction
Done
Predicting
Done
Writing output
Done


In [62]:
test =  pd.read_csv('predictions_test.csv')

In [63]:
test.head(10)

,Unnamed: 0,amazon-id,Awesome
0,0,-3827781478900123339,1
1,1,-1146746308871643825,0
2,2,7943165581472649995,1
3,3,9108673782042818101,1
4,4,6254356016991899485,1
5,5,-5853727546265815229,1
6,6,7870026881680799188,1
7,7,-9080174460600991411,1
8,8,5049399051352648854,1
9,9,2944373440361513822,1


In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

ct_opts = [('reviewText_bow', TfidfVectorizer(max_features = 4000), 'reviewText'),
          ('summary_bow', TfidfVectorizer(max_features = 4000), 'summary'),
          ('salesRank_norm', MinMaxScaler(), ['salesRank']),
          ('price_norm', MinMaxScaler(), ['price'])]
ct = ColumnTransformer(ct_opts, remainder = 'drop')

In [65]:
train_new = prep_df(train)

In [67]:
combined_df = train_new.groupby('amazon-id').agg({'reviewText': ' '.join, 'summary': ' '.join, 'salesRank': lambda x: np.mean(x), 'price': lambda x: np.mean(x)})

In [68]:
combined_df.head(10)

,reviewText,summary,salesRank,price
amazon-id,,,,
-9217723718720870868,year album aretha franklin released one best a...,come back album one ree best love classy album...,1310516,9.90
-9215746463819797371,one beautiful instrumental cd ever heard beaut...,amazing piano beautiful album mainlypiano high...,309139,48.11
-9213978596308513604,july celebrate anniversary release epic souths...,ok great,280309,15.86
-9211290576571923870,first heard recording released lp spell ever s...,key joyous laughter mediterranean sunshine,321654,17.37
-9208769561690910545,another rare case score film better actual mov...,great soundtrack halloween resurrection evil f...,17515,12.00
-9208198150317071838,another excellent collection song josh ritter ...,excellent josh ritter excellent songwriter per...,114996,33.61
-9207623668807116759,album compilation los cuates de sinaloa greate...,los cuates de sinaloa breaking bad azul,221487,31.27
-9203846742259231188,like creative beat eclectic break unkle dj sha...,another unkle classic essential unkle cool unk...,553883,48.40
-9202734100323412002,superb composition wonderful unique composer p...,exclellent azerbaijani jazz glorious,405958,15.08


In [74]:
train_new['sent_scores']= train_new['reviewText'].apply(lambda review: sid.polarity_scores(review))

In [72]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [71]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/isuruabeysekara/nltk_data...


True

In [75]:
train_new['compound']  = train_new['sent_scores'].apply(lambda score_dict: score_dict['compound'])

In [81]:
train_new.head(10)

,reviewerID,amazon-id,helpful,unixReviewTime,reviewText,overall,reviewTime,summary,price,categories,...,first-release-year,songs,salesRank,related,sent_scores,compound,comp_score,sum_sent_scores,sum_compound,sum_comp_score
0,-4984057859803657856,1877521326299865484,"[2, 2]",1302739200,nice music practicing tai chi downloaded phone...,4,"04 14, 2011",beautiful,16.47,"['CDs & Vinyl', 'New Age']",...,NaN,"[7058439142327364074, 6037075874942075284, 852...",27222,"{'also_bought': [-404470919165672227, 11968160...","{'neg': 0.124, 'neu': 0.604, 'pos': 0.272, 'co...",0.7089,pos,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.5994,pos
1,9136764282801708742,1877521326299865484,"[11, 11]",1180396800,recently starting tai chi love immensily addin...,5,"05 29, 2007",tranquillity motion,16.47,"['CDs & Vinyl', 'New Age']",...,NaN,"[7058439142327364074, 6037075874942075284, 852...",27222,"{'also_bought': [-404470919165672227, 11968160...","{'neg': 0.025, 'neu': 0.472, 'pos': 0.503, 'co...",0.9877,pos,"{'neg': 0.0, 'neu': 0.263, 'pos': 0.737, 'comp...",0.4215,pos
2,2164551966908582519,1877521326299865484,"[0, 0]",1361404800,wife us class room kid love loved price great ...,5,"02 21, 2013",great stuff,16.47,"['CDs & Vinyl', 'New Age']",...,NaN,"[7058439142327364074, 6037075874942075284, 852...",27222,"{'also_bought': [-404470919165672227, 11968160...","{'neg': 0.0, 'neu': 0.317, 'pos': 0.683, 'comp...",0.9442,pos,"{'neg': 0.0, 'neu': 0.196, 'pos': 0.804, 'comp...",0.6249,pos
3,-7309200698931694843,1877521326299865484,"[4, 4]",1338163200,bought music go dr lam dvd music perfect give ...,5,"05 28, 2012",beautiful,16.47,"['CDs & Vinyl', 'New Age']",...,NaN,"[7058439142327364074, 6037075874942075284, 852...",27222,"{'also_bought': [-404470919165672227, 11968160...","{'neg': 0.0, 'neu': 0.597, 'pos': 0.403, 'comp...",0.8957,pos,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.5994,pos
4,-4461682407031037732,1877521326299865484,"[0, 0]",1396310400,help exercise set proper mood happy quality pl...,5,"04 1, 2014",tai chi music,16.47,"['CDs & Vinyl', 'New Age']",...,NaN,"[7058439142327364074, 6037075874942075284, 852...",27222,"{'also_bought': [-404470919165672227, 11968160...","{'neg': 0.0, 'neu': 0.462, 'pos': 0.538, 'comp...",0.8519,pos,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
5,-7459674557577458961,1877521326299865484,"[0, 0]",1287273600,tai chi music paul lam cd exactly expected mus...,4,"10 17, 2010",tai chi music,16.47,"['CDs & Vinyl', 'New Age']",...,NaN,"[7058439142327364074, 6037075874942075284, 852...",27222,"{'also_bought': [-404470919165672227, 11968160...","{'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound...",0.6124,pos,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
6,8261414655084321703,1877521326299865484,"[15, 30]",1107043200,highly admire work paul lam tai chi extremely ...,1,"01 30, 2005",love dr lam work sample available,16.47,"['CDs & Vinyl', 'New Age']",...,NaN,"[7058439142327364074, 6037075874942075284, 852...",27222,"{'also_bought': [-404470919165672227, 11968160...","{'neg': 0.227, 'neu': 0.56, 'pos': 0.213, 'com...",-0.1027,neg,"{'neg': 0.0, 'neu': 0.543, 'pos': 0.457, 'comp...",0.6369,pos
7,3180071471897091081,1877521326299865484,"[0, 0]",1395705600,husband getting year previous experience pract...,5,"03 25, 2014",love dvd,16.47,"['CDs & Vinyl', 'New Age']",...,NaN,"[7058439142327364074, 6037075874942075284, 852...",27222,"{'also_bought': [-404470919165672227, 11968160...","{'neg': 0.0, 'neu': 0.757, 'pos': 0.243, 'comp...",0.8979,pos,"{'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'comp...",0.6369,pos
8,1423705408082526473,1877521326299865484,"[1, 1]",1379808000,tried tai chi yoga music fitting provides righ...,5,"09 22, 2013",ideal music,16.47,"['CDs & Vinyl', 'New Age']",...,NaN,"[7058439142327364074, 6037075874942075284, 852...",27222,"{'also_bought': [-404470919165672227, 11968160...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos,"{'neg': 0.0, 'neu': 0.227, 'pos':

In [77]:
train_new['comp_score'] = train_new['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

In [79]:
train_new['sum_sent_scores']= train_new['summary'].apply(lambda review: sid.polarity_scores(review))

In [80]:
train_new['sum_compound']  = train_new['sum_sent_scores'].apply(lambda score_dict: score_dict['compound'])
train_new['sum_comp_score'] = train_new['sum_compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

In [89]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer



def get_sentiment_scores(data):
    
    sid = SentimentIntensityAnalyzer()
    
    data['review_sent_scores'] = data['reviewText'].apply(lambda review: sid.polarity_scores(str(review)))
    data['sum_sent_scores']= data['summary'].apply(lambda summary: sid.polarity_scores(str(summary)))
     
    data['review_compound']  = train_new['review_sent_scores'].apply(lambda score_dict: score_dict['compound'])    
    data['sum_compound']  = data['sum_sent_scores'].apply(lambda score_dict: score_dict['compound'])
    
    data['review_comp_score'] = train_new['review_compound'].apply(lambda c: 'pos' if c >=0 else 'neg')
    data['sum_comp_score'] = data['sum_compound'].apply(lambda c: 'pos' if c >=0 else 'neg')
    
    return data
    
    

In [91]:
awe_vec = awesome_products(train_new)

In [92]:
print(awe_vec)

amazon-id
-9217723718720870868    0
-9215746463819797371    1
-9213978596308513604    0
-9211290576571923870    0
-9208769561690910545    0
-9208198150317071838    1
-9207623668807116759    1
-9203846742259231188    0
-9202734100323412002    1
-9198322104429679287    0
-9196816809534721621    1
-9195787036214670721    0
-9195715895034041804    0
-9185450485401072051    0
-9185028087385392712    1
-9184754614868031288    0
-9181938828093623181    0
-9177871146610584170    0
-9175284111494784505    1
-9171391896663756617    1
-9171133252298665868    0
-9170624073749960483    0
-9168022306965593241    0
-9164485698573154140    1
-9163887239523687605    1
-9163211980141980025    1
-9163010614846223784    0
-9162759304190308282    0
-9161926972130019940    0
-9157117359304219671    1
                       ..
 9182231784957840868    0
 9183110027749334191    0
 9183152728540863858    0
 9183587774518525868    1
 9184679746257273506    1
 9187454525560420789    1
 9191699717616638912    1
 9

In [93]:
awe_vec.value_counts()

1    6181
0    4362
Name: overall, dtype: int64

In [21]:
from imblearn.over_sampling import SMOTE


def correct_oversampling(x_train, y_train):
    print("Fixing imbalance of class samples")
    sm = SMOTE(random_state=27)
    x_train_new, y_train_new = sm.fit_resample(x_train, y_train)
    print("Done")
    return x_train_new, y_train_new